In [27]:
import pandas as pd
import numpy as np
import os

In [28]:
data_dir = '../data'
img_ext = ['jpg', 'png', 'jpeg']

segmentation = ['face', 'body', 'clothing', 'environment']

In [29]:
image_list = []
for image in os.listdir(f'{data_dir}/images_ps'):
    if image.split('.')[1] in img_ext:
        image_list.append(image)

len(image_list)

1126

In [30]:
# Initialize dataframe
df = pd.read_csv(f'{data_dir}/annotation_data/face/data_faceIdentity.csv').set_index('imageName')

columns = []
for i in segmentation:
    for file in os.listdir(f'{data_dir}/annotation_data/{i}'):
        filename = file.split('.')[0]
        columns.append(filename)

all_features = pd.DataFrame(index=df.index, columns=columns)
# all_features

In [31]:
# put data into initialized dataframe and save it
for i in segmentation:
    for file in os.listdir(f'{data_dir}/annotation_data/{i}'):
        filename = file.split('.')[0]
        df = pd.read_csv(f'{data_dir}/annotation_data/{i}/{file}').set_index('imageName')
        for j in df.index:
            all_features.loc[j, filename] = list(df.loc[j])

all_features.to_csv('../data/annotation_data/data_all.csv')

In [32]:
# define function to get the aggregated value of the ratings
def get_aggregated_value(filename):
    trait_name = filename.split('/')[-1].split('_')[0]
    df = pd.read_csv(filename).set_index('imageName')
    sum = df.sum(axis=1)
    count = df.count(axis=1)
    df = pd.DataFrame({'sum':sum, 'count':count})
    df[f'{trait_name}'] = df['sum'] / df['count']
    df = df.get([f'{trait_name}'])
    return df

In [33]:
# goes through the data for all ratings and add the aggregated value for each image to a list
traits = []
trait_dir = '../data/rawdata_trait_rating/'
for trait_csv in os.listdir('../data/rawdata_trait_rating/'):
    traits.append(get_aggregated_value(trait_dir + trait_csv))

# create dataframe from the aggregated values of each image and then put them into one column training target
traits = pd.concat(traits, axis=1)
new_column = str(list(traits.columns))
traits[new_column] = traits.apply(lambda row: [
    row['attractive'], row['competent'], row['dominant'], row['feminine'], row['open'], row['warm'], row['youthful']
], axis=1)
traits = traits.get([new_column])
# traits

In [34]:
# combine the training data and the training target
all_data = all_features.merge(traits, left_index=True, right_index=True)

# Define a function to convert list elements to floats
def convert_list_elements_to_float(cell):
    if isinstance(cell, list):
        return np.array(cell, dtype=np.float32)
    return cell

# Apply the function to each cell in the DataFrame
all_data = all_data.map(convert_list_elements_to_float)
all_data

,data_faceIdentity,data_gaze,data_action,data_headOrientation,data_jointDistance,data_shape,data_clothColor,data_hairColor,data_environmentColor,data_gist,"['attractive', 'competent', 'dominant', 'feminine', 'open', 'warm', 'youthful']"
imageName,,,,,,,,,,,
obj365_train_000000121119_000.jpg,"[0.12895739, -0.028168134, 0.00757975, -0.0497...","[0.0, 1.0, 0.0, 0.0, 0.0, 0.0]","[0.002333118, 0.009564623, 0.010592721, -0.011...","[7.5218234, -1.1987422, 19.710966]","[0.20098007, 0.3210551, 0.5550481, 0.7843166, ...","[0.5017561, 0.49716136, -0.5002197, 0.5061276,...","[28.738771, 130.58502, 125.413605]","[1.0, 1.0, 1.0, 1.5, 3.0]","[144.05965, 129.26022, 131.55663]","[-0.00831051, -0.017807305, -0.007456646, -0.0...","[2.1190476, 4.6666665, 4.2380953, 1.4761904, 3..."
COCO_train2014_000000015645.jpg,"[0.02622014, 0.081596665, 0.09578178, 0.043360...","[0.0, 1.0, 0.0, 0.0, 0.0, 0.0]","[-0.008572297, -0.007843425, 0.012377907, -0.0...","[-45.380337, -14.898459, 3.2193282]","[0.20062017, 0.22529441, 0.40198928, 0.3694324...","[0.5239638, 0.51242507, -0.49369422, 0.5200673...","[183.39679, 140.82408, 139.4729]","[1.25, 2.75, 1.0, 2.0, 1.25]","[89.59149, 155.37447, 159.07156]","[-0.011086742, 0.002311628, 0.00432519, 0.0044...","[4.9761906, 5.452381, 5.2619047, 6.404762, 4.7..."
openimages_9375bad4f7fb7f07_000.jpg,"[-0.024200823, 0.033819914, 0.023400877, -0.02...","[0.0, 0.0, 1.0, 0.0, 0.0, 0.0]","[-0.007622262, 0.00956685, 0.012877201, -0.010...","[-0.49005643, 18.24955, 10.881761]","[0.20261985, 0.36788186, 0.5587657, 0.5890628,...","[0.50715315, 0.49351236, -0.50082695, 0.504786...","[97.35879, 128.20456, 126.32881]","[1.0, 2.5, 1.0, 2.75, 1.0]","[111.57404, 128.76472, 136.01012]","[-0.008220214, -0.014275488, 0.001812318, -0.0...","[5.418605, 4.604651, 3.1860466, 6.395349, 3.90..."
obj365_train_000000152196_000.jpg,"[-0.001752781, 0.10471826, 0.09139394, 0.00050...","[1.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[-0.022864908, -0.009293311, 0.014308339, -0.0...","[-5.290164, -10.876716, -13.168068]","[0.20239152, 0.29074258, 0.5197746, 0.74333066...","[0.49157432, 0.491557, -0.4998648, 0.5013863, ...","[138.12492, 136.22755, 137.46814]","[1.25, 1.75, 1.0, 3.0, 1.0]","[81.21457, 129.12741, 125.6436]","[0.002911689, -0.017169828, 0.01690869, -0.016...","[5.142857, 4.642857, 3.7380953, 6.6904764, 4.0..."
obj365_train_000000124687_000_v2.jpg,"[0.063750975, 0.024633992, 0.018427502, 0.0305...","[1.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[-0.016813006, -0.012196642, 0.008504863, -0.0...","[-2.1394632, -7.2794085, 22.632244]","[0.1983144, 0.31450784, 0.5302256, 0.72155076,...","[0.5069542, 0.5005446, -0.48763558, 0.50750524...","[176.008, 136.23224, 121.116394]","[1.5, 1.5, 1.0, 1.5, 2.75]","[127.2692, 141.75041, 136.37422]","[0.003828755, -0.001846732, 0.001435783, -0.02...","[2.139535, 3.6046512, 3.2790697, 1.3023256, 3...."
...,...,...,...,...,...,...,...,...,...,...,...
openimages_760d6765c8fd2994_000.jpg,"[-0.06154379, -0.013045238, 0.022959623, 0.004...","[1.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[-0.019059326, -0.000775224, 0.019176897, -0.0...","[-4.4105678, -10.12891, -0.82248086]","[0.20381983, 0.32736084, 0.56379765, 0.5398477...","[0.5094925, 0.49926317, -0.5049064, 0.5057019,...","[140.48985, 129.8249, 125.32335]","[3.0, 1.0, 1.0, 1.0, 1.0]","[118.817726, 133.3955, 133.32318]","[0.01392939, 0.000987544, -0.021520052, -0.036...","[2.4186046, 4.7906976, 3.5581396, 5.4418607, 3..."
obj365_train_000000295170_000.jpg,"[0.055740464, 0.04205254, -0.025778953, -0.045...","[0.0, 1.0, 0.0, 0.0, 0.0, 0.0]","[-0.025736548, -0.01261155, 0.002416412, -0.01...","[-2.0477452, -2.0441082, -10.286333]","[0.20338818, 0.32070142, 0.48857722, 0.5789143...","[0.50178576, 0.50216603, -0.4984893, 0.5135028...","[98.35711, 130.07988, 120.39369]","[1.25, 1.0, 1.0, 1.0, 2.5]","[148.10095, 126.485374, 132.80249]","[0.008089812, 0.011577478, -0.012705069, -0.01...","[2.4651163, 4.581395, 5.023256, 1.3953488, 3.2..."
obj365_train_000000634686_000.jpg,"[0.042224776, 0.07374723, 0.109102

In [61]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

# seperate the training data and the target label
X = all_data.get(list(all_data.columns)[:-1])

# dataframe to store scaled data
scaled_X = pd.DataFrame(index=X.index)

# apply Min-Max scaling to training data
for i, feature in enumerate(X):
    scaler = MinMaxScaler()
    stacked = np.vstack(X[feature])
    scaled = scaler.fit_transform(stacked)
    unstacked = [list(row) for row in scaled]
    scaled_X[feature] = unstacked

# check index sequence
assert(scaled_X.index.all()==y.index.all())

### need to create labels (similar/dissmilar) for data pairs.

In [62]:
from sklearn.metrics.pairwise import cosine_similarity

def cosine_sim(a, b):
    a = np.array(a).reshape(1, -1)
    b = np.array(b).reshape(1, -1)
    return cosine_similarity(a, b)[0][0]

In [63]:
# get the labels
y = all_data.assign(output=all_data.get([list(all_data.columns)[-1]])).get(['output'])

# concatenate all the features together into one list
scaled_X['input'] = scaled_X.apply(lambda row: np.array(sum([row.iloc[i] for i in range(len(row))], [])), axis=1)
scaled_all = scaled_X.get(['input']).merge(y, left_index=True, right_index=True).reset_index()
scaled_all

,imageName,input,output
0,obj365_train_000000121119_000.jpg,"[0.92500126, 0.3313901, 0.48801482, 0.3691098,...","[2.1190476, 4.6666665, 4.2380953, 1.4761904, 3..."
1,COCO_train2014_000000015645.jpg,"[0.5208456, 0.74441016, 0.82499117, 0.7367672,...","[4.9761906, 5.452381, 5.2619047, 6.404762, 4.7..."
2,openimages_9375bad4f7fb7f07_000.jpg,"[0.3224957, 0.56463706, 0.54845953, 0.4655692,...","[5.418605, 4.604651, 3.1860466, 6.395349, 3.90..."
3,obj365_train_000000152196_000.jpg,"[0.41080356, 0.8314114, 0.8082274, 0.56753117,...","[5.142857, 4.642857, 3.7380953, 6.6904764, 4.0..."
4,obj365_train_000000124687_000_v2.jpg,"[0.66848725, 0.5300725, 0.5294587, 0.68622065,...","[2.139535, 3.6046512, 3.2790697, 1.3023256, 3...."
...,...,...,...
1120,openimages_760d6765c8fd2994_000.jpg,"[0.17559308, 0.38829413, 0.54677373, 0.5846894...","[2.4186046, 4.7906976, 3.5581396, 5.4418607, 3..."
1121,obj365_train_000000295170_000.jpg,"[0.63697493, 0.59561455, 0.36056772, 0.3848691...","[2.4651163, 4.581395, 5.023256, 1.3953488, 3.2..."
1122,obj365_train_000000634686_000.jpg,"[0.58380586, 0.7148745, 0.8758831, 0.69453996,...","[1.8837209, 4.2619047, 3.3953488, 1.6046512, 4..."
1123,COCO_val2014_000000509192.jpg,"[0.17919587, 0.6266786, 0.23679179, 0.39790532...","[3.511628, 4.5116277, 3.5581396, 6.0465117, 3...."


In [64]:
# Initialize a list to store the pairs
pairs = []

# Iterate through each possible pair of rows
for i in range(len(scaled_all)):
    for j in range(i + 1, len(scaled_all)):
        row1 = scaled_all.iloc[i]
        row2 = scaled_all.iloc[j]
        pairs.append({
            'imageNames': (row1['imageName'],row2['imageName']),
            'input': (row1['input'],row2['input']),
            'output': (row1['output'],row2['output'])
        })

# Convert the list of pairs to a dataframe
pairs_df = pd.DataFrame(pairs)

pairs_df.to_csv('../data/all_image_pairs.csv')

# Display the new dataframe
pairs_df

,imageNames,input,output
0,"(obj365_train_000000121119_000.jpg, COCO_train...","([0.92500126, 0.3313901, 0.48801482, 0.3691098...","([2.1190476, 4.6666665, 4.2380953, 1.4761904, ..."
1,"(obj365_train_000000121119_000.jpg, openimages...","([0.92500126, 0.3313901, 0.48801482, 0.3691098...","([2.1190476, 4.6666665, 4.2380953, 1.4761904, ..."
2,"(obj365_train_000000121119_000.jpg, obj365_tra...","([0.92500126, 0.3313901, 0.48801482, 0.3691098...","([2.1190476, 4.6666665, 4.2380953, 1.4761904, ..."
3,"(obj365_train_000000121119_000.jpg, obj365_tra...","([0.92500126, 0.3313901, 0.48801482, 0.3691098...","([2.1190476, 4.6666665, 4.2380953, 1.4761904, ..."
4,"(obj365_train_000000121119_000.jpg, COCO_train...","([0.92500126, 0.3313901, 0.48801482, 0.3691098...","([2.1190476, 4.6666665, 4.2380953, 1.4761904, ..."
...,...,...,...
632245,"(obj365_train_000000295170_000.jpg, COCO_val20...","([0.63697493, 0.59561455, 0.36056772, 0.384869...","([2.4651163, 4.581395, 5.023256, 1.3953488, 3...."
632246,"(obj365_train_000000295170_000.jpg, COCO_train...","([0.63697493, 0.59561455, 0.36056772, 0.384869...","([2.4651163, 4.581395, 5.023256, 1.3953488, 3...."
632247,"(obj365_train_000000634686_000.jpg, COCO_val20...","([0.58380586, 0.7148745, 0.8758831, 0.69453996...","([1.8837209, 4.2619047, 3.3953488, 1.6046512, ..."
632248,"(obj365_train_000000634686_000.jpg, COCO_train...","([0.58380586, 0.7148745, 0.8758831, 0.69453996...","([1.8837209, 4.2619047, 3.3953488, 1.6046512, ..."


In [10]:
# # split the training and testing set
# X_train, X_test, y_train, y_test = train_test_split(scaled_X, y, test_size=0.2, random_state=42)
# X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size=0.5, random_state=42)

# # concatenate all the features together into one list
# X_train['concatenated'] = X_train.apply(lambda row: np.array(sum([row.iloc[i] for i in range(len(row))], [])), axis=1)
# X_val['concatenated'] = X_val.apply(lambda row: np.array(sum([row.iloc[i] for i in range(len(row))], [])), axis=1)
# X_test['concatenated'] = X_test.apply(lambda row: np.array(sum([row.iloc[i] for i in range(len(row))], [])), axis=1)

# # get only the concatenated features
# X_train = X_train.get('concatenated')
# X_val = X_val.get('concatenated')
# X_test_label = X_test.get('concatenated')

# # get only the ratings
# y_train = y_train.get(y_train.columns[0])
# y_val = y_val.get(y_val.columns[0])
# y_test_label = y_test.get(y_test.columns[0])

# # convert everything to float ndarray
# X_train = np.vstack(X_train)
# X_val = np.vstack(X_val)
# X_test = np.vstack(X_test_label)

# # convert everything to float ndarray
# y_train = np.vstack(y_train)
# y_val = np.vstack(y_val)
# y_test = np.vstack(y_test_label)

In [11]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset

# Check if GPU is available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

Using device: cuda


In [12]:
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
X_val_tensor = torch.tensor(X_val, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32).to(device)
y_val_tensor = torch.tensor(y_val, dtype=torch.float32).to(device)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32).to(device)

# create DataLoader
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
val_dataset = TensorDataset(X_val_tensor, y_val_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [13]:
# model definition
class NeuralNet(nn.Module):
    def __init__(self, input_size, output_size):
        super(NeuralNet, self).__init__()
        self.fc1 = nn.Linear(input_size, 128)
        self.bn1 = nn.BatchNorm1d(128)
        self.dropout1 = nn.Dropout(0.5)
        
        self.fc2 = nn.Linear(128, 64)
        self.bn2 = nn.BatchNorm1d(64)
        self.dropout2 = nn.Dropout(0.5)
        
        self.fc3 = nn.Linear(64, 32)
        self.bn3 = nn.BatchNorm1d(32)
        self.dropout3 = nn.Dropout(0.5)
        
        self.fc4 = nn.Linear(32, output_size)
    
    def forward(self, x):
        x = torch.relu(self.bn1(self.fc1(x)))
        x = self.dropout1(x)
        
        x = torch.relu(self.bn2(self.fc2(x)))
        x = self.dropout2(x)
        
        x = torch.relu(self.bn3(self.fc3(x)))
        x = self.dropout3(x)
        
        x = self.fc4(x)
        return x

# model definition
class NeuralNet2(nn.Module):
    def __init__(self, input_size, output_size):
        super(NeuralNet2, self).__init__()
        self.fc1 = nn.Linear(input_size, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, output_size)
    
    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    
def weights_init(m):
    if isinstance(m, nn.Linear):
        nn.init.xavier_uniform_(m.weight)
        nn.init.zeros_(m.bias)

In [14]:
# Siamese Network Definition
class SiameseNetwork(nn.Module):
    def __init__(self, input_size, output_size):
        super(SiameseNetwork, self).__init__()
        self.shared_net = NeuralNet2(input_size, output_size)

    def forward_once(self, x):
        return self.shared_net(x)

    def forward(self, input1, input2):
        output1 = self.forward_once(input1)
        output2 = self.forward_once(input2)
        return output1, output2

# Contrastive Loss Definition
class ContrastiveLoss(nn.Module):
    def __init__(self, margin=1.0):
        super(ContrastiveLoss, self).__init__()
        self.margin = margin

    def forward(self, output1, output2, label):
        euclidean_distance = F.pairwise_distance(output1, output2)
        loss_contrastive = torch.mean((1-label) * torch.pow(euclidean_distance, 2) +
                                      (label) * torch.pow(torch.clamp(self.margin - euclidean_distance, min=0.0), 2))
        return loss_contrastive

In [15]:
# Instantiate the model, loss function, and optimizer
input_size = X_train_tensor.shape[1]
output_size = y_train_tensor.shape[1]

In [16]:
model = SiameseNetwork(input_size, output_size).to(device)
model.apply(weights_init)
criterion = ContrastiveLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
# Add learning rate scheduler
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=30)
prev_lr = optimizer.param_groups[0]['lr']
best_loss = float('inf')  # Initialize to infinity

# Training loop
num_epochs = 500

# Early stopping parameters
early_stop_patience = 100
no_improvement_count = 0

for epoch in range(num_epochs):
    model.train()
    epoch_loss = 0
    for X_batch, y_batch in train_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)

        X_batch_1, X_batch_2 = X_batch[:X_batch.shape[0]//2], X_batch[X_batch.shape[0]//2:]
        X_batch_1 = X_batch_1.to(device)
        X_batch_2 = X_batch_2.to(device)
        # y_batch_1, y_batch_2 = y_batch[:y_batch.shape[0]//2], y_batch[y_batch.shape[0]//2:]
        labels = torch.randint(0, 2, (X_batch.shape[0]//2,)).to(device)  # Example labels (0: different, 1: similar)
        
        output1, output2 = model(X_batch_1, X_batch_2)
        loss = criterion(output1, output2, labels)
        optimizer.zero_grad()
        loss.backward()
        # torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()
        
        epoch_loss += loss.item()
    
    training_loss = epoch_loss / len(train_loader)

    # Validation step
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for X_batch, y_batch in val_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)

            X_batch_1, X_batch_2 = X_batch[:X_batch.shape[0]//2].to(device), X_batch[X_batch.shape[0]//2:].to(device)
            y_batch_1, y_batch_2 = y_batch[:y_batch.shape[0]//2], y_batch[y_batch.shape[0]//2:]
            labels = torch.randint(0, 2, (X_batch.shape[0]//2,)).to(device)  # Example labels (0: different, 1: similar)

            output1, output2 = model(X_batch_1, X_batch_2)
            loss = criterion(output1, output2, labels)
            val_loss += loss.item()
    
    validation_loss = val_loss / len(val_loader)

    if (epoch + 1) % 20 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Training Loss: {training_loss:.4f}, Validation loss: {validation_loss}')

    # Save the model if it has the best loss so far
    if validation_loss < best_loss:
        best_loss = validation_loss
        torch.save(model.state_dict(), 'best_model_siamese.pth')
        no_improvement_count = 0  # Reset counter
    else:
        no_improvement_count += 1
    
    # Step the scheduler
    scheduler.step(validation_loss)
    
    # Early stopping
    if no_improvement_count >= early_stop_patience:
        print(f"Early stopping at epoch {epoch+1}")
        break

Epoch [20/500], Training Loss: 0.2719, Validation loss: 0.23856769502162933
Epoch [40/500], Training Loss: 0.2700, Validation loss: 0.29110442847013474
Epoch [60/500], Training Loss: 0.2835, Validation loss: 0.29173924028873444
Epoch [80/500], Training Loss: 0.2779, Validation loss: 0.28698649629950523
Epoch [100/500], Training Loss: 0.2739, Validation loss: 0.2808472290635109
Epoch [120/500], Training Loss: 0.2627, Validation loss: 0.26927024871110916
Epoch [140/500], Training Loss: 0.2677, Validation loss: 0.3036064878106117
Epoch [160/500], Training Loss: 0.2541, Validation loss: 0.2695915587246418
Early stopping at epoch 173


In [17]:
# import torch
# import torch.nn.functional as F
# from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_curve, auc
# import numpy as np

# # Assume siamese_net is your trained Siamese network
# # input1, input2, and pair_labels are your test data and labels

# # Forward pass
# output1, output2 = siamese_net(input1, input2)

# # Calculate Euclidean distances
# distances = F.pairwise_distance(output1, output2)

# # Determine an optimal threshold
# # For simplicity, we can use a fixed threshold (e.g., 0.5), but in practice, you might want to tune this
# threshold = 0.5

# # Predict labels based on the threshold
# predicted_labels = (distances < threshold).float()

# # Convert tensors to numpy arrays for metric calculation
# pair_labels_np = pair_labels.cpu().numpy()
# predicted_labels_np = predicted_labels.cpu().numpy()

# # Calculate evaluation metrics
# accuracy = accuracy_score(pair_labels_np, predicted_labels_np)
# precision = precision_score(pair_labels_np, predicted_labels_np)
# recall = recall_score(pair_labels_np, predicted_labels_np)
# f1 = f1_score(pair_labels_np, predicted_labels_np)

# # Calculate ROC curve and AUC
# fpr, tpr, _ = roc_curve(pair_labels_np, distances.cpu().numpy())
# roc_auc = auc(fpr, tpr)

# # Print metrics
# print(f'Accuracy: {accuracy:.4f}')
# print(f'Precision: {precision:.4f}')
# print(f'Recall: {recall:.4f}')
# print(f'F1-Score: {f1:.4f}')
# print(f'AUC: {roc_auc:.4f}')

# # Plot ROC curve
# import matplotlib.pyplot as plt

# plt.figure()
# plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (area = {roc_auc:.4f})')
# plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
# plt.xlim([0.0, 1.0])
# plt.ylim([0.0, 1.05])
# plt.xlabel('False Positive Rate')
# plt.ylabel('True Positive Rate')
# plt.title('Receiver Operating Characteristic')
# plt.legend(loc='lower right')
# plt.show()


Ideal Loss: 0.01~0.1225